# Lab: Refactoring for Better Class Design

Objectives


1.   Improve cohesion by ensuring each class has a single, clear responsibility.
2.   Reduce coupling to minimize dependencies between classes.
3. Apply refactoring techniques to make the code more maintainable.
4. Use Representation-Driven Design (RDD) by utilizing private variables and controlled access.







In [ ]:
class GameCharacter:
    def __init__(self, name, hp, attack, defense, gold, inventory, level, experience, quests):
        self.name = name
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.gold = gold
        self.inventory = inventory
        self.level = level
        self.experience = experience
        self.quests = quests

    def attack_enemy(self, enemy):
        damage = self.attack - enemy.defense
        if damage > 0:
            enemy.hp -= damage
        print(f"{self.name} attacks {enemy.name} for {damage} damage!")

    def buy_item(self, item, price):
        if self.gold >= price:
            self.gold -= price
            self.inventory.append(item)
            print(f"{self.name} bought {item}!")
        else:
            print(f"{self.name} doesn't have enough gold!")

    def display_status(self):
        print(f"Name: {self.name}, HP: {self.hp}, Attack: {self.attack}, Defense: {self.defense}, Gold: {self.gold}, Level: {self.level}, Experience: {self.experience}")

    def gain_experience(self, points):
        self.experience += points
        if self.experience >= 100:
            self.level_up()

    def level_up(self):
        self.level += 1
        self.attack += 5
        self.defense += 3
        self.hp += 10
        print(f"{self.name} leveled up to level {self.level}!")

    def complete_quest(self, quest):
        if quest not in self.quests:
            self.quests.append(quest)
            print(f"{self.name} completed quest: {quest}!")
        else:
            print(f"{self.name} already completed this quest.")

    def buy_armor(self, armor, price):
        if self.gold >= price:
            self.gold -= price
            self.inventory.append(armor)
            print(f"{self.name} bought armor: {armor}!")
        else:
            print(f"{self.name} doesn't have enough gold for armor!")

    def use_potion(self, potion):
        if potion in self.inventory:
            self.hp += 20
            self.inventory.remove(potion)
            print(f"{self.name} used a {potion} potion!")
        else:
            print(f"{self.name} doesn't have a {potion} potion!")


What's the problem of the code above?

In [ ]:
#Answer here

#==================================================================
#  🔥 ปัญหาของโค้ดนี้ 🔥
#==================================================================

# ❌ ไม่มีความปลอดภัยของโค้ด ไม่มีการทำ RDD
#    - การตั้งชื่อตัวแปรไม่ปลอดภัยและการรับค่าที่มากเกินไปควรมีค่าเริ่มต้น

# ❌ OOP ไม่เต็มที่ inventory และ quests อยู่ในคลาสเดียวกัน
#    - มันควรถูกแยกออกไปให้เหมาะกับหน้าที่ของมัน ไม่ใช่ยัดไว้ใน GameCharacter

# ❌ Bad Class Design คลาส GameCharacter รับผิดชอบหลายหน้าที่เกินไป
#    - ทั้งโจมตี ซื้อของ ทำเควส เลเวลอัพ ฯลฯ มันควรแยกออกเป็นคลาสย่อย เช่น `InventoryManager`, `QuestManager`

# ❌ ข้อมูลสำคัญเช่น hp, attack, gold ถูกเปิดเผยให้แก้ไขได้โดยตรงจากภายนอก
#    - ควรใช้ getter และ setter เพื่อป้องกันการเข้าถึงโดยตรง *เสริมจากข้อแรก

# ❌ ค่าต่างๆ เช่น การเพิ่ม attack, defense เมื่อเลื่อนเลเวล ควรจัดการในรูปแบบที่ยืดหยุ่น
#    - ไม่ควร hardcode ค่าตายตัว *ตามสไลด์หน้าที่ 12
#    - ฟังก์ชันไม่ครบ ไม่มีรับดาเมจ

# ❌ ไม่มีการจัดการข้อผิดพลาดในกรณีที่ตัวละครหรือศัตรูมีค่า hp <= 0 หรือกรณีอื่นๆ
#    - ตัวละครยังโจมตีศัตรูที่ตายไปแล้วได้ หรืออาจเกิด error ได้ ควรเช็คก่อน

# ❌ [QOL] ไม่มีข้อความแสดงเมื่อ inventory หรือ quest ว่าง
#    - ควรมีเพื่อแจ้งให้ทราบว่าทำงานปกติและทำให้ผู้ใช้งานรู้ว่าตอนนี้ inventory หรือ quest ว่าง QOL นั้นแหละครับ

# ❌ ฟังก์ชัน buy_item และ buy_armor มีการทำงานที่เหมือนๆกัน
#    - ควรรวมเป็นฟังก์ชันเดียว ไม่ก็อยู่ในคลาสเดียว

# ❌ [QOL] ฟังก์ชัน gain_experience ไม่มีการจัดการกรณีที่ประสบการณ์เกิน 100 XP แล้วจะเลื่อนเลเวลหลายครั้ง
#    - ควรทำให้รองรับการอัปเลเวลแบบต่อเนื่องถ้า XP เกิน

# ❌ [QOL] ถ้าโจมตีแล้วไม่มีดาเมจ (ค่า damage <= 0)
#    - แสดงข้อความโจมตีปกติซึ่งควรเป็นข้อความอื่นจะดีกว่า เช่น "การโจมตีไร้ผล!"

# ❌ ฟังก์ชัน use_potion จะเพิ่ม HP ให้ผู้เล่น 20 โดยไม่สนใจจำนวนของ potion ใน inventory
#    - ใช้ได้ไม่มีวันหมด ควรเช็คว่ามี potion อยู่ใน inventory ก่อน

#หมายเหตุ QOL = Quality of life
#==================================================================


What's your solution to modify the code?

Hint: The modified version should contain 3 - 4 classes

In [ ]:
# 🔥 วิธีแก้! 🔥

# ✅ เปลี่ยนชื่อตัวแปรให้ปลอดภัยตามหลักการ Encapsulation

# ✅ ทำให้โค้ดและคลาสถูกต้องตาม Responsibility-Driven Design (RDD)

# ✅ แยก `InventoryManager` ออกมาให้จัดการไอเทมโดยเฉพาะ

# ✅ สร้างคลาส `Item` แทนการใช้แค่ชื่อไอเทมเป็น string

# ✅ แยก `QuestManager` ออกมาให้ดูแลระบบเควสแทน GameCharacter

# ✅ `Shop` ควรดูแลระบบการซื้อของ แยกจาก GameCharacter

# ✅ `GameCharacter` จะโฟกัสแค่ค่าสถานะของตัวละคร

# ✅ ใช้ Getter และ Setter เพื่อป้องกันการแก้ไขค่าโดยตรง

# ✅ ปรับปรุงระบบโจมตี
#    - สร้างคลาสดูแลระบบต่อสู้ ArenaManager
#    - ถ้าดาเมจ <= 0 ให้แสดงว่า "การโจมตีไร้ผล!"
#    - ถ้าศัตรู HP <= 0 ห้ามโจมตีอีก

# ✅ ปรับปรุงการเลเวลอัปให้รองรับ XP เกิน 100 ได้

# ✅ `use_potion()` ควรลดจำนวน potion ใน inventory เมื่อใช้งาน

# ✅ เพิ่มฟังก์ชัน `display_status()` ให้แสดง inventory และ quest ด้วย

# ✅ รวมฟังก์ชัน `buy_item()` และ `buy_armor()` เป็นฟังก์ชันเดียว
#    - ให้ Shop จัดการระบบซื้อของทั้งหมดแทน

# หมายเหตุ * มันคือการบอกแบบคล่าวๆในโค้ดอาจจะมากหรือน้อยกว่านี้

Refactor the code using good class design principle.

In [ ]:
from rich.console import Console
from rich.text import Text

global console
global color1
global color2
global color3
global color4
global banner
console = Console()
color1 = "#CB9DF0"
color2 = "#F0C1E1"
color3 = "#FDDBBB"
color4 = "#FFF9BF"
banner = "═" * 50

class GameCharacter:
    def __init__(self, name):
        self.__name = str(name)
        self.__health = 100
        self.__max_health = self.__health
        self.__attack = 0
        self.__defense = 0
        self.__gold = 0
        self.__inventory = InventoryManager()
        self.__level = 1
        self.__experience = 0
        self.__max_exp = self.__level * 100
        self.__quests = []

    def get_name(self):
        return self.__name

    def set_name(self, name):
        self.__name = str(name)

    def get_health(self):
        return self.__health

    def set_health(self, health):
        self.__health = max(0, health)

    def get_max_health(self):
        return self.__max_health

    def set_max_health(self, new_max):
        self.__attack = max(self.__health, new_max)

    def get_attack(self):
        return self.__attack

    def set_attack(self, attack):
        self.__attack = max(0, attack)

    def get_defense(self):
        return self.__defense

    def set_defense(self, defense):
        self.__defense = max(0, defense)

    def get_gold(self):
        return self.__gold

    def set_gold(self, gold):
        self.__gold = max(0, gold)

    def get_inventory(self):
        return self.__inventory

    def set_inventory(self, inv):
        if not isinstance(inv, InventoryManager):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__inventory = inv

    def get_level(self):
        return self.__level

    def set_level(self, level):
        self.__level = max(1, level)

    def get_experience(self):
        return self.__experience

    def set_experience(self, experience):
        self.__experience = max(0, experience)

    def get_max_exp(self):
        return self.__max_exp

    def set_max_exp(self, max_exp):
        self.__max_exp = max(self.__level * 100, max_exp)

    def get_quests(self):
        return self.__quests

    def set_quests(self, quests):
        if type(quests) != type([]):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__quests = quests

    def display_status(self):
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print("            🌟 Character Information 🌟            ")
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"Name         : {self.__name}")
        print(f"Health       : {self.__health} HP")
        print(f"Attack       : {self.__attack} ATK")
        print(f"Defense      : {self.__defense} DEF")
        print(f"Gold         : {self.__gold} G")
        print(f"Inventory    : ⫷{self.__inventory.get_item_inv()}|⫸")
        print(f"Level        : {self.__level}")
        print(f"Experience   : {self.__experience}/{self.__max_exp} XP")
        print(f"Quests       : {', '.join(self.__quests) if self.__quests else 'No active quests'}")
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

    def gain_experience(self, points):
        self.__experience += points
        while self.__experience >= self.__max_exp:
            self.__experience -= self.__max_exp
            self.__level_up()
            self.__max_exp = self.__level * 100

    def __level_up(self):
        self.__level += 1
        self.__attack += 5
        self.__defense += 3
        self.__health += 10
        self.__max_health += 10
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"        🌟 {self.__name} Level UP! [ level {self.__level} ]🌟    ")
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

    def pick_item(self, item):
        if not isinstance(item, Item):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__inventory.add_item(item)

    def use_item(self, item):
        for items in self.__inventory.get_inventory():
            if str(item) == items.get_Item_name():
                items.Use(self)

class InventoryManager:
    def __init__(self):
        self.__inventors = []
        self.__space = 64

    def get_inventory(self):
        return self.__inventors

    def set_inventory(self, new_inv):
        if type(new_inv) != type([]):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__inventors = new_inv

    def get_space(self):
        return self.__space

    def set_space(self, new_space):
        if type(new_space) != type(0):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__space = new_space

    def add_space(self, amount):
        if type(amount) != type(0):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__space += amount

    def get_item_inv(self):
        items = ""
        for item in self.__inventors:
            items += "|"+item.get_Item_name()
        return items

    def add_item(self, item):
        if len(self.__inventors) >= self.__space:
            print("ช่องในกระเป๋าไม่เพียงพอ")
            return
        if not isinstance(item, Item):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__inventors.append(item)

    def remove_item(self, item):
        for items in self.__inventors:
            if item == items:
                self.__inventors.remove(items)
                return

class Item:
    def __init__(self, name, _type, ability, effect=0):
        self.__item_name = str(name)
        self.__item_type = str(_type)
        if self.__item_type.lower() == "potion" :
            if str(ability).lower() in ["heal","buff_atk","buff_def"]:
                self.__item_ability = ability
                if self.__item_ability == "heal":
                    self.__item_effect = 20
                elif self.__item_ability == "buff_atk":
                    self.__item_effect = 50
                elif self.__item_ability == "buff_def":
                    self.__item_effect = 40
                return
        elif self.__item_type.lower() == "armor" :
            if str(ability).lower() in ["protect"]:
                self.__item_ability = ability
                self.__item_effect = effect
                return
        self.__item_ability = "Unknown"

    def get_Item_name(self):
        return self.__item_name

    def set_Item_name(self,  new_name):
        self.__item_name = str(new_name)

    def get_Item_type(self):
        return self.__item_type

    def set_Item_type(self, new_type):
        self.__item_type = str(new_type)

    def get_Item_ability(self):
        return self.__item_ability

    def set_Item_ability(self, new_ability):
        self.__item_ability = str(new_ability)

    def get_Item_effect(self):
        return self.__item_effect

    def set_Item_effect(self, new_effect):
        if type(new_effect) != type(0):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__item_effect = new_effect

    def Use(self, player):
        if self.__item_type.lower() == "potion":
            if self.__item_ability == "heal":
                if (player.get_max_health() - player.get_health()) >= self.__item_effect:
                    player.set_health(player.get_health() + self.__item_effect)
                else:
                    player.set_health(player.get_max_health())
                player.get_inventory().remove_item(self)
            elif self.__item_ability == "buff_atk":
                player.set_attack(player.get_attack() + self.__item_effect)
                player.get_inventory().remove_item(self)
            elif self.__item_ability == "buff_def":
                player.set_defense(player.get_defense() + self.__item_effect)
                player.get_inventory().remove_item(self)
        elif self.__item_type.lower() == "armor":
            if self.__item_ability == "protect":
                player.set_defense(player.get_defense() + self.__item_effect)
                player.get_inventory().remove_item(self)
        else:
            return

class Shop:
    def __init__(self, name):
        self.__store_name = str(name)
        self.__store_storage = {}

    def get_Store_name(self):
        return self.__store_name

    def set_Store_name(self, new_name):
        self.__store_name = str(new_name)

    def get_Store_storage(self):
        return self.__store_storage

    def set_Store_storage(self, new_storage):
        if type(new_storage) != type({}):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        self.__store_storage = new_storage

    def show_shop(self):
        console.print(Text(f"\n╔═{banner}╗", style=f"bold {color1}"))
        console.print(Text(f"{f'🌈 ⋆⁺₊⋆ Welcome to {self.__store_name} ⋆⁺₊⋆ 🌈'.center(52)}", style=f"bold {color2}"))
        console.print(Text(f"╚{banner}═╝\n", style=f"bold {color1}"))
        console.print("📜 Items for sale:\n", style=f"bold underline {color3}")

        for index, (item, price) in enumerate(self.__store_storage.items(), start=1):
            console.print(f"  [italic #F0C1E1]{index:2}[/italic #F0C1E1]. | {item.get_Item_type():<7} | [#D1E9F6]{item.get_Item_name():<20}[/#D1E9F6] | [#FDDBBB]{price:>5} [/#FDDBBB][#FFF9BF]G.[/#FFF9BF]")

        console.print(Text(f"\n\n╚{banner}═╝\n", style=f"bold {color1}"))
        print(" (◕‿◕)ﾉ*:･ﾟ✧ What would you like to buy?")

    def add_Item_storage(self, item, price):
        if not isinstance(item, Item):
            print("⚠︎ Type ของตัวแปรไม่ถูกต้อง")
            return
        for items in self.__store_storage:
            if items.get_Item_name() == item.get_Item_name():
                print("⚠︎ มีไอเทมนี้แล้ว")
                return
        self.__store_storage.update({item : price})

    def Buy(self, player, item_name):
        console.print(Text(f"\n╔═{banner}╗", style=f"bold {color1}"))
        console.print(Text(f"{f'🌈 ⋆⁺₊⋆ {self.__store_name} ⋆⁺₊⋆ 🌈'.center(52)}", style=f"bold {color2}"))
        console.print(Text(f"╚{banner}═╝\n", style=f"bold {color1}"))

        for item, price in self.__store_storage.items():
            if item_name == item.get_Item_name():
                console.print(f"  | {item.get_Item_type():<7} | [#D1E9F6]{item_name:<20}[/#D1E9F6] | [#FDDBBB]{price:>5} [/#FDDBBB][#FFF9BF]G.[/#FFF9BF]")
                player_item_buy =[item, price]
            else:
                console.print(f"[#FF8A8A]No Item.[/#FF8A8A].")
                return

        console.print(f"[bold #F5DAD2]\n /)/)\n( . .)\n( づ♡[/bold #F5DAD2] [bold #FBFBFB] Do you want to buy this item?[/bold #FBFBFB] ( [#CCE0AC]Yes[/#CCE0AC] or [#FF8A8A]No[/#FF8A8A] ).")
        num = input(">> ").lower()
        if num == "yes":
            if player.get_gold() >= player_item_buy[1]:
                player.set_gold(player.get_gold() - player_item_buy[1])
                player.pick_item(player_item_buy[0])
                console.print(f"[bold #F5DAD2]\n /)/)\n( . .)\n( づ♡[/bold #F5DAD2] [bold #F7CFD8] Thank you very much![/bold #F7CFD8]")
            else:
                console.print(f"[#FF8A8A]You No Gold[/#FF8A8A].")
        elif num == "no":
            console.print(f"[#CCE0AC]OK[/#CCE0AC].")
        else:
            console.print(f"[#FF8A8A]Please type Yes or No.[/#FF8A8A].")

class QuestManager:
    def __init__(self, name, description, reward):
        self.__quest_name = str(name)
        self.__quest_description = str(description)
        if type(reward) != type(0): self.__quest_reward = 0
        self.__quest_reward = reward
        self.__quest_status = False

    def get_quest_name(self):
        return self.__quest_name

    def set_quest_name(self, name):
        self.__quest_name = str(name)

    def get_quest_description(self):
        return self.__quest_description

    def set_quest_description(self, description):
        self.__quest_description = str(description)

    def get_quest_reward(self):
        return self.__quest_reward

    def set_quest_reward(self, reward):
        if type(reward) != type(0): return
        self.__quest_reward = reward

    def get_quest_status(self):
        return self.__quest_status

    def set_quest_status(self, status):
        if status not in [True,False]: return
        self.__quest_status = status

    def complete_quest(self, player):
        if not self.__quest_status:
            player.set_gold(player.get_gold() + self.__quest_reward)
            self.__quest_status = True
            print(f"Quest '{self.__quest_name}' completed! Reward: {self.__quest_reward} gold.")
        else:
            print(f"Quest '{self.__quest_name}' is already completed.")

class ArenaManager:
    def __init__(self, player, enemy):
        self.__player = player
        self.__enemy = enemy

    def attack(self, attacker, defender):
        damage = self.__cal_damage(attacker.get_attack(), defender.get_defense())
        defender.set_health(defender.get_health() - damage)
        print(f"{attacker.get_name()} attacks {defender.get_name()} for {damage} damage!")

    def __cal_damage(self, ATK, DEF):
        return max(1, ATK - DEF)

    def start_battle(self):
        banner = "═════════════════════════"
        color1 = "rgb(9, 18, 44)"  # #09122C
        color2 = "rgb(135, 35, 65)"  # #872341
        color3 = "rgb(190, 49, 68)"  # #BE3144
        color4 = "rgb(231, 117, 100)"  # #E17564

        console.print(Text(f"\n╔═{banner}╗", style=f"bold {color1}"))
        console.print(Text(f"{f'⚔️🌟 Battle Start! 🛡️✨'.center(20)}", style=f"bold {color2}"))
        console.print(Text(f"╚{banner}═╝\n", style=f"bold {color1}"))

        while self.__player.get_health() > 0 and self.__enemy.get_health() > 0:
            console.print(Text(f"{self.__player.get_name()} attacks {self.__enemy.get_name()}!", style=f"{color3}"))
            self.attack(self.__player, self.__enemy)

            if self.__enemy.get_health() <= 0:
                console.print(Text(f"💥 {self.__enemy.get_name()} has been defeated! ✨", style=f"{color4}"))
                break

            console.print(Text(f"{self.__enemy.get_name()} strikes back at {self.__player.get_name()}!", style=f"{color3}"))
            self.attack(self.__enemy, self.__player)

            if self.__player.get_health() <= 0:
                console.print(Text(f"⚔️ {self.__player.get_name()} is defeated! 💔", style=f"{color4}"))
                break

            console.print(Text(f"\n🔥 {self.__player.get_name()} HP: {self.__player.get_health()} | {self.__enemy.get_name()} HP: {self.__enemy.get_health()}", style=f"{color2}"))

        console.print(Text("⚔️ Battle Ended! ⚔️", style=f"bold {color1}"))

import random

def generate_random_item():
        names = ["Health Potion"]
        categories = ["potion"]
        effects = ["heal"]

        random_name = random.choice(names)
        random_category = random.choice(categories)
        random_effect = random.choice(effects)

        return Item(random_name, random_category, random_effect)